In [1]:
!pip install gym
!pip install pygame
!pip install tf_agents==0.15.0
!pip install tensorflow


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import gym
import numpy as np
import pygame
import time
import random
from gym import spaces
from gym.envs.registration import register
import matplotlib.pyplot as plt
import tensorflow as tf
from tf_agents.agents.dqn import dqn_agent
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import sequential
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.specs import tensor_spec
from tf_agents.utils import common

# player piece colors [player1, player2]
colors = ['#FF0000', '#0000FF']

# paths for each tile 1-35
node_combinations = [ 
    [(0,3), (1,5), (2,6), (4,7)], [(0,4), (1,5), (2,6), (3,7)], [(0,3), (1,6), (2,5), (4,7)], [(0,6), (1,5), (2,4), (3,7)],
    [(0,1), (2,3), (4,5), (6,7)], [(0,4), (1,5), (2,3), (6,7)], [(0,6), (1,5), (2,3), (4,7)], [(0,5), (1,4), (2,7), (3,6)],
    [(0,5), (1,4), (2,6), (3,7)], [(0,3), (1,4), (2,5), (6,7)], [(0,6), (1,4), (2,5), (3,7)], [(0,5), (1,4), (2,3), (6,7)],
    [(0,2), (1,3), (4,6), (5,7)], [(0,2), (1,3), (4,5), (6,7)], [(0,5), (1,3), (2,7), (4,6)], [(0,6), (1,3), (2,7), (4,5)],
    [(0,4), (1,3), (2,6), (5,7)], [(0,5), (1,3), (2,6), (4,7)], [(0,4), (1,3), (2,5), (6,7)], [(0,6), (1,3), (2,5), (4,7)],
    [(0,5), (1,3), (2,4), (6,7)], [(0,6), (1,3), (2,4), (5,7)], [(0,3), (1,2), (4,7), (5,6)], [(0,3), (1,2), (4,6), (5,7)],
    [(0,3), (1,2), (4,5), (6,7)], [(0,4), (1,2), (3,7), (5,6)], [(0,5), (1,2), (3,7), (4,6)], [(0,6), (1,2), (3,7), (4,5)],
    [(0,4), (1,2), (3,6), (5,7)], [(0,5), (1,2), (3,6), (4,7)], [(0,4), (1,2), (3,5), (6,7)], [(0,6), (1,2), (3,5), (4,7)],
    [(0,5), (1,2), (3,4), (6,7)], [(0,6), (1,2), (3,4), (5,7)], [(0,7), (1,2), (3,4), (5,6)]
]


In [3]:
class Tile():
    def __init__(self, tile_num, tile_connections):
        self.tile_num = tile_num
        self.image = pygame.image.load("TsuroImages/" + str(tile_num) + ".png")
        self.image = pygame.transform.scale(self.image, (100, 100))
        self.tile_connections = tile_connections
        self.rotation = 1
    
    def move(self, current_node):
        next_node = 0
        next_player_tile = 0
        for connection in self.tile_connections:
            if current_node in connection:
                n1, n2 = connection
                if n1 == current_node:
                    next_node, next_player_tile, next_x, next_y = self.new_tile_node(n2)
                else:
                    next_node, next_player_tile, next_x, next_y = self.new_tile_node(n1)
                return next_node, next_player_tile, next_x, next_y
        raise Exception("Issue in moving players")
    
    # update number of times rotation should be applied to connections and image
    def rotate_tile(self, rotate):
        self.image = pygame.transform.rotate(self.image, rotate * -90)
        self.tile_connections = [tuple((element + (2 * rotate)) % 8 for element in couple ) for couple in self.tile_connections]
        self.rotation = 1 if (self.rotation + 1 % 4 == 0) else self.rotation + 1
        
    def get_rotation(self):
        return self.rotation
    
    def new_tile_node(self, current_node):
        next_node = 0
        next_x = 0
        next_y = 0
        next_player_tile = 0
        match current_node:
            case 0:
                next_node = 3
                next_player_tile = -1
                next_x = -1
            case 1:
                next_node = 6
                next_player_tile = -6
                next_y = -1
            case 2:
                next_node = 5
                next_player_tile = -6
                next_y = -1
            case 3:
                next_node = 0
                next_player_tile = 1
                next_x = 1
            case 4:
                next_node = 7
                next_player_tile = 1
                next_x = 1
            case 5:
                next_node = 2
                next_player_tile = 6
                next_y = 1
            case 6:
                next_node = 1
                next_player_tile = 6
                next_y = 1
            case 7:
                next_node = 4
                next_player_tile = -1
                next_x = -1
            case _:
                raise Exception("Issue in tile board")
                
        return next_node, next_player_tile, next_x, next_y
    

In [4]:
class TsuroEnv(gym.Env):
    #
    def __init__(self):
        self.screen = None
        self.current_player = 1
        self.num_tiles = 35
        self.tile_board_size = (6, 6)
        self.rotation_board_size = (6, 6)
        self.player_board_size = (36,8)
        self.num_players = 2
        self.tiles = []
        for i in range(self.num_tiles):
            self.tiles.append(Tile(i, node_combinations[i]))
            
        self.remaining_tiles = []
        for i in range(self.num_tiles):
            self.remaining_tiles.append(i)
        random.shuffle(self.remaining_tiles)

        self.remaining_players = []
        for i in range(self.num_players):
            self.remaining_players.append(i+1)
        
        self.player_tiles = []
        for i in range(self.num_players):
            player_tiles = []
            for i in range(3):
                player_tiles.append(self.remaining_tiles.pop())
            self.player_tiles.append(player_tiles)
            
        self.rotation_board = np.zeros(self.rotation_board_size, dtype = int)
        self.tile_board = np.zeros(self.tile_board_size, dtype = int)
        self.player_board = np.zeros(self.player_board_size, dtype = int)

        self.action_space = spaces.Discrete(11)
        self.observation_space = spaces.Box(low=-1, high=35, shape=(363,))
        
    # Resets the environment to default state
    def reset(self): 
        self.current_player = 1
        self.rotation_board = np.zeros(self.rotation_board_size, dtype = int)
        self.tile_board = np.zeros(self.tile_board_size, dtype = int)
        self.player_board = np.zeros(self.player_board_size, dtype = int)
        
        self.tiles = []
        for i in range(self.num_tiles):
            self.tiles.append(Tile(i, node_combinations[i]))
            
        self.remaining_tiles = []
        for i in range(self.num_tiles):
            self.remaining_tiles.append(i)
        random.shuffle(self.remaining_tiles)
            
        self.remaining_players = []
        for i in range(self.num_players):
            self.remaining_players.append(i+1)
            
        self.player_tiles = []
        for i in range(self.num_players):
            player_tiles = []
            for i in range(3):
                player_tiles.append(self.remaining_tiles.pop())
            self.player_tiles.append(player_tiles)
            
        #########################################
        #TODO: TESTING STUFF TO BE REMOVED LATER#
        #########################################
        for i in range(self.num_players):
            self.player_board[random.randint(0,5)][i+1] = i+1
            
        initial_obs = np.hstack((self.player_tiles[self.current_player - 1], self.player_board.flatten(), self.tile_board.flatten(), self.rotation_board.flatten()))

        return initial_obs
    
    # Makes a move in the game based on inputs from player or AI
    def step(self, action = -2, move = -1):
        if move == -1:
            card, rotate = self.get_card(action)
        else:
            card = move
            rotate = 0
        
        action = self.player_tiles[self.current_player - 1][card]
        
        if action == -1:
            observation = np.hstack((self.player_tiles[self.current_player - 1], self.player_board.flatten(), self.tile_board.flatten(), self.rotation_board.flatten()))
            reward = -1
            done = 0
            return observation, reward, done, {}
            
        # Removes used tile and adds new tile from deck to hand
        self.player_tiles[self.current_player-1].remove(action)
        if len(self.remaining_tiles) > 0:
            self.player_tiles[self.current_player-1].append(self.remaining_tiles.pop())
        else:
            self.player_tiles[self.current_player-1].append(-1)
        
        # Rotates tile (Only used by AI)
        self.tiles[action].rotate_tile(rotate)
        
        reward = 0
        self.place_tile(action+1)
        self.move_players()
        reward = self.reward_function()
        if self.game_is_over():
            done = 1
        else:
            done = 0
        self.current_player = self.next_player()
        observation = np.hstack((self.player_tiles[self.current_player - 1], self.player_board.flatten(), self.tile_board.flatten(), self.rotation_board.flatten()))
        return observation, reward, done, {}
    
    # Decides if the game is over
    def game_is_over(self):
        if len(self.remaining_players) <= 1:
            return True
        return False
    
    # Decides the reward (Only used for AI)
    def reward_function(self):
        if not self.game_is_over():
            return 1
        if self.game_is_over() and self.current_player in self.remaining_players:
            return 2
        return -1
    
    # Places tile in self.tile_board
    def place_tile(self, tile):
        tile_number, node_number = np.where(self.player_board == self.current_player)
        x, y = TsuroEnv.euclidean_division(self, tile_number)
        x = x[0]
        y = y[0]
        self.tile_board[x][y] += tile
        self.rotation_board[x][y] += self.tiles[tile-1].get_rotation()
    
    # Moves player piece in self.player_board
    def move_players(self):
        for player in self.remaining_players:
            tile_number, node_number = np.where(self.player_board == player)
            x, y = TsuroEnv.euclidean_division(self, tile_number)
            x = x[0]
            y = y[0]
            while self.tile_board[x][y] != 0:
                tile = self.tiles[(self.tile_board[x][y])-1]
                next_node, next_player_tile, next_x, next_y = tile.move(node_number)
                self.player_board[tile_number[0]][node_number[0]] = 0
                if ((tile_number[0] % 6 == 0) and ((tile_number[0] + next_player_tile) % 6 == 5)) or (tile_number[0] + next_player_tile < 0) or (tile_number[0] + next_player_tile > 35) or ((tile_number[0] % 6 == 5) and ((tile_number[0] + next_player_tile) % 6 == 0)):
                    self.remaining_players.remove(player)
                    break
                else:
                    self.player_board[tile_number[0] + next_player_tile][next_node] = player
                    x += next_x
                    y += next_y
                    tile_number, node_number = np.where(self.player_board == player)
    
    # Quotient and Remainder
    def euclidean_division(self, x, y = 6):
        return x % y, x // y
    
    # Action (card, rotation) from input
    def get_card(self, x, y = 4):
        return  x // y, x % y

    # Decide whos turn it is
    def next_player(self):
        if len(self.remaining_players) == 0:
            return -1
        if self.current_player not in self.remaining_players:
            for player in self.remaining_players:
                if player > self.current_player:
                    return player
                else:
                    return self.remaining_players[0]
        return self.remaining_players[(self.remaining_players.index(self.current_player) + 1) % len(self.remaining_players)]
    
    def get_state(self):
        observation = np.hstack((self.player_tiles[self.current_player - 1], self.player_board.flatten(), self.tile_board.flatten(), self.rotation_board.flatten()))
        return observation
        
    # Render the environment
    def render(self, mode):
        screen = pygame.display.set_mode((650, 750))
        screen.fill((255, 255, 255))

        # Draw the game board
        board = pygame.image.load("TsuroImages/board.png")
        board = pygame.transform.scale(board, (600, 600))
        screen.blit(board, (25,25))
        
        # Draw current players hand
        for i in range (len(self.player_tiles[self.current_player-1])):
            tile = self.player_tiles[self.current_player-1][i]
            screen.blit(self.tiles[tile].image, (75 + (i * 200), 635))
        
        # Draw the tiles on the board
        for x in range(self.tile_board_size[0]):
            for y in range(self.tile_board_size[1]):
                val = self.tile_board[x][y]
                if val != 0:
                    tile = self.tiles[val-1]
                    screen.blit(tile.image, (25 + x * 100, 25 + y * 100))
                    
        # Draw the players' pieces on the board
        for i in self.remaining_players:
            tile_number, node_number = np.where(self.player_board == i)
            y_add = 0
            x_add = 0
            y_mult = 0
            x_mult = 0
            
            match node_number[0]:
                case 0:
                    y_add = 35
                case 1:
                    x_add = 35
                case 2:
                    x_add = 70
                case 3:
                    x_add = 100
                    y_add = 35
                case 4:
                    x_add = 100
                    y_add = 70
                case 5:
                    x_add = 70
                    y_add = 100
                case 6:
                    x_add = 35
                    y_add = 100
                case 7:
                     y_add = 70
                case _:
                    raise Exception("Issue in drawing the player board")
                    
            if tile_number[0] != 0:
                x_mult, y_mult = TsuroEnv.euclidean_division(self, tile_number[0])
            
            pygame.draw.circle(screen, colors[i-1], (25 + x_add + (100 * x_mult), 25 + y_add + (100 * y_mult)), 5)
            
        # Draw text to show who won when game is over
        if self.game_is_over() or self.current_player == -1:
            font = pygame.font.Font('freesansbold.ttf', 32)
            text = font.render('Player ' + str(self.current_player) + ' wins', True, '#00FF00')
            textRect = text.get_rect()
            textRect.center = (650 // 2, 750 // 2)
            screen.blit(text, textRect)
            
        pygame.display.update()



In [5]:
num_iterations = 2500000

initial_collect_steps = 5000
collect_steps_per_iteration = 1
replay_buffer_max_length = 100000

batch_size = 100
learning_rate = 1e-3
log_interval = 200

num_eval_episodes = 20
eval_interval = 5000

register(
    id='TsuroEnv',
    entry_point=TsuroEnv,
)

env_name = "TsuroEnv"

train_py_env = suite_gym.load(env_name)
eval_py_env = suite_gym.load(env_name)

train_env = tf_py_environment.TFPyEnvironment(train_py_env)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

fc_layer_params = (200, 100)
action_tensor_spec = tensor_spec.from_spec(train_env.action_spec())
num_actions = action_tensor_spec.maximum - action_tensor_spec.minimum + 1

def dense_layer(num_units):
    return tf.keras.layers.Dense(num_units, activation=tf.keras.activations.relu, kernel_initializer=tf.keras.initializers.VarianceScaling(scale=2.0, mode='fan_in', distribution='truncated_normal'))

dense_layers = [dense_layer(num_units) for num_units in fc_layer_params]
q_values_layer = tf.keras.layers.Dense(num_actions, activation=None, kernel_initializer=tf.keras.initializers.RandomUniform(minval=-0.03, maxval=0.03), bias_initializer=tf.keras.initializers.Constant(-0.2))
q_net = sequential.Sequential(dense_layers + [q_values_layer])

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

train_step_counter = tf.Variable(0)

agent = dqn_agent.DqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent.initialize()

eval_policy = agent.policy
collect_policy = agent.collect_policy

random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(),
                                                train_env.action_spec())

replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=train_env.batch_size,
    max_length=replay_buffer_max_length)
    
def compute_avg_return(environment, policy, num_episodes=10):

    total_return = 0.0
    for _ in range(num_episodes):

        time_step = environment.reset()
        episode_return = 0.0

        while not time_step.is_last():
            action_step = policy.action(time_step)
            time_step = environment.step(action_step.action)
            episode_return += time_step.reward
        total_return += episode_return

    avg_return = total_return / num_episodes
    return avg_return.numpy()[0]

def collect_step(environment, policy, buffer):
    time_step = environment.current_time_step()
    action_step = policy.action(time_step)
    next_time_step = environment.step(action_step)
    # commenting out render will make training quicker
    # environment.render("human")
    traj = trajectory.from_transition(time_step, action_step, next_time_step)
    buffer.add_batch(traj)

def collect_data(env, policy, buffer, steps):
    for _ in range(steps):
        collect_step(env, policy, buffer)
        
train_checkpointer = common.Checkpointer(
    ckpt_dir="Checkpoints1/",
    max_to_keep=1,
    agent=agent,
    policy=agent.policy,
    replay_buffer=replay_buffer,
    global_step=train_step_counter
)

train_checkpointer.initialize_or_restore()


In [6]:
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3, 
    sample_batch_size=batch_size, 
    num_steps=2).prefetch(3)

iterator = iter(dataset)

agent.train = common.function(agent.train)

returns = []

with open("Checkpoints1/returns.txt", "r") as txt:
    for line in txt:
        returns.append(line)
        
for i in range(len(returns)):
    returns[i] = returns[i].strip()
    returns[i] = float(returns[i])

Instructions for updating:
Use `tf.data.Dataset.counter(...)` instead.


Instructions for updating:
Use `tf.data.Dataset.counter(...)` instead.


Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [7]:
# Initial Collection for new agent
'''
collect_data(train_env, random_policy, replay_buffer, initial_collect_steps)
agent.train_step_counter.assign(0)

avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
returns = [avg_return]
'''

'\ncollect_data(train_env, random_policy, replay_buffer, initial_collect_steps)\nagent.train_step_counter.assign(0)\n\navg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)\nreturns = [avg_return]\n'

In [ ]:
for _ in range(num_iterations):
    collect_data(train_env, agent.collect_policy, replay_buffer, collect_steps_per_iteration)

    experience, unused_info = next(iterator)
    train_loss = agent.train(experience).loss

    step = agent.train_step_counter.numpy()

    if step % log_interval == 0:
        print('step = {0}: loss = {1}'.format(step, train_loss))

    if step % eval_interval == 0:
        avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
        print('step = {0}: Average Return = {1}'.format(step, avg_return))
        returns.append(avg_return)
        train_checkpointer.save(train_step_counter)
        with open("Checkpoints1/returns.txt", "w") as txt:
            for item in returns:
                txt.write(str(item) + "\n")

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))


Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))


step = 4910200: loss = 4.242068767547607
step = 4910400: loss = 3.6957173347473145
step = 4910600: loss = 3.7417309284210205
step = 4910800: loss = 3.8654563426971436
step = 4911000: loss = 3.522522449493408
step = 4911200: loss = 3.467543363571167
step = 4911400: loss = 4.57596492767334
step = 4911600: loss = 4.736881732940674
step = 4911800: loss = 3.9961531162261963
step = 4912000: loss = 3.5100462436676025
step = 4912200: loss = 4.569034099578857
step = 4912400: loss = 3.286997079849243
step = 4912600: loss = 4.12788200378418
step = 4912800: loss = 4.19862699508667
step = 4913000: loss = 4.36104679107666
step = 4913200: loss = 3.366215705871582
step = 4913400: loss = 4.484061241149902
step = 4913600: loss = 4.552647113800049
step = 4913800: loss = 4.136657238006592
step = 4914000: loss = 3.7621123790740967
step = 4914200: loss = 4.79841947555542
step = 4914400: loss = 4.7251996994018555
step = 4914600: loss = 3.1707098484039307
step = 4914800: loss = 3.3531203269958496
step = 49150

step = 4948400: loss = 3.099590301513672
step = 4948600: loss = 3.913886070251465
step = 4948800: loss = 4.322664260864258
step = 4949000: loss = 4.226951599121094
step = 4949200: loss = 4.665009498596191
step = 4949400: loss = 4.375265598297119
step = 4949600: loss = 3.373652458190918
step = 4949800: loss = 4.382088661193848
step = 4950000: loss = 3.111623764038086
step = 4950000: Average Return = 2.25
step = 4950200: loss = 2.81516170501709
step = 4950400: loss = 2.462397336959839
step = 4950600: loss = 5.184128284454346
step = 4950800: loss = 3.0734071731567383
step = 4951000: loss = 4.610994815826416
step = 4951200: loss = 4.7414021492004395
step = 4951400: loss = 3.3588039875030518
step = 4951600: loss = 5.238795280456543
step = 4951800: loss = 3.855922222137451
step = 4952000: loss = 3.487271547317505
step = 4952200: loss = 4.44066047668457
step = 4952400: loss = 4.326478004455566
step = 4952600: loss = 3.5620105266571045
step = 4952800: loss = 2.7313895225524902
step = 4953000: 

step = 4986600: loss = 3.9881768226623535
step = 4986800: loss = 3.301422595977783
step = 4987000: loss = 3.7611634731292725
step = 4987200: loss = 3.4207894802093506
step = 4987400: loss = 4.18076229095459
step = 4987600: loss = 3.899148464202881
step = 4987800: loss = 4.796546936035156
step = 4988000: loss = 3.312188148498535
step = 4988200: loss = 3.652630567550659
step = 4988400: loss = 4.0162353515625
step = 4988600: loss = 4.4349589347839355
step = 4988800: loss = 4.213515758514404
step = 4989000: loss = 4.571341514587402
step = 4989200: loss = 3.1246681213378906
step = 4989400: loss = 3.8849334716796875
step = 4989600: loss = 2.753955125808716
step = 4989800: loss = 4.366640090942383
step = 4990000: loss = 3.5524849891662598
step = 4990000: Average Return = 3.4000000953674316
step = 4990200: loss = 4.844635963439941
step = 4990400: loss = 4.674066543579102
step = 4990600: loss = 3.750828266143799
step = 4990800: loss = 3.9779491424560547
step = 4991000: loss = 3.7350542545318604

...

step = 6279000: loss = 2.8336246013641357
step = 6279200: loss = 4.341048717498779
step = 6279400: loss = 4.535970211029053
step = 6279600: loss = 3.252981424331665
step = 6279800: loss = 4.196390151977539
step = 6280000: loss = 3.363233804702759
step = 6280000: Average Return = 3.299999952316284
step = 6280200: loss = 3.8546974658966064
step = 6280400: loss = 3.3904051780700684
step = 6280600: loss = 3.02980637550354
step = 6280800: loss = 3.6686160564422607
step = 6281000: loss = 3.2842793464660645
step = 6281200: loss = 4.994241714477539
step = 6281400: loss = 2.8416991233825684
step = 6281600: loss = 4.707734107971191
step = 6281800: loss = 5.198725700378418
step = 6282000: loss = 3.243246078491211
step = 6282200: loss = 3.8905229568481445
step = 6282400: loss = 4.225902557373047
step = 6282600: loss = 3.3357560634613037
step = 6282800: loss = 2.8557422161102295
step = 6283000: loss = 4.670887470245361
step = 6283200: loss = 3.3889567852020264
step = 6283400: loss = 3.9924998283386

step = 6316600: loss = 4.542575359344482
step = 6316800: loss = 3.5240397453308105
step = 6317000: loss = 3.98862886428833
step = 6317200: loss = 2.725816249847412
step = 6317400: loss = 4.578917026519775
step = 6317600: loss = 3.1153008937835693
step = 6317800: loss = 4.148638725280762
step = 6318000: loss = 3.901007652282715
step = 6318200: loss = 4.207172870635986
step = 6318400: loss = 6.258719444274902
step = 6318600: loss = 3.620839834213257
step = 6318800: loss = 3.3239033222198486
step = 6319000: loss = 4.489083290100098
step = 6319200: loss = 3.8810834884643555
step = 6319400: loss = 2.5742013454437256
step = 6319600: loss = 3.527594566345215
step = 6319800: loss = 3.556354522705078
step = 6320000: loss = 3.367035150527954
step = 6320000: Average Return = 2.450000047683716
step = 6320200: loss = 3.098568916320801
step = 6320400: loss = 3.7828691005706787
step = 6320600: loss = 4.4021100997924805
step = 6320800: loss = 4.561493396759033
step = 6321000: loss = 2.719778060913086


step = 6354600: loss = 2.839888334274292
step = 6354800: loss = 4.561308860778809
step = 6355000: loss = 4.070228576660156
step = 6355000: Average Return = 3.799999952316284
step = 6355200: loss = 3.4362664222717285
step = 6355400: loss = 3.819904088973999
step = 6355600: loss = 3.8827462196350098
step = 6355800: loss = 3.5644874572753906
step = 6356000: loss = 3.658891201019287
step = 6356200: loss = 4.027995586395264
step = 6356400: loss = 2.8364593982696533
step = 6356600: loss = 4.59271240234375
step = 6356800: loss = 4.0071516036987305
step = 6357000: loss = 4.975329399108887
step = 6357200: loss = 3.718966007232666
step = 6357400: loss = 2.5153326988220215
step = 6357600: loss = 4.01694393157959
step = 6357800: loss = 3.4677867889404297
step = 6358000: loss = 3.2129745483398438
step = 6358200: loss = 3.427034854888916
step = 6358400: loss = 3.343395233154297
step = 6358600: loss = 3.7774670124053955
step = 6358800: loss = 4.0924482345581055
step = 6359000: loss = 4.49902009963989

step = 6392400: loss = 2.91021466255188
step = 6392600: loss = 3.4500012397766113
step = 6392800: loss = 3.954021453857422
step = 6393000: loss = 3.9212286472320557
step = 6393200: loss = 2.492276191711426
step = 6393400: loss = 4.197525978088379
step = 6393600: loss = 5.115484714508057
step = 6393800: loss = 3.796456813812256
step = 6394000: loss = 3.6982085704803467
step = 6394200: loss = 4.363584518432617
step = 6394400: loss = 3.9249043464660645
step = 6394600: loss = 4.277926921844482
step = 6394800: loss = 4.832815170288086
step = 6395000: loss = 4.777397155761719
step = 6395000: Average Return = 2.950000047683716
step = 6395200: loss = 4.407007694244385
step = 6395400: loss = 3.544520139694214
step = 6395600: loss = 3.7875280380249023
step = 6395800: loss = 5.119160175323486
step = 6396000: loss = 4.932976722717285
step = 6396200: loss = 2.970798969268799
step = 6396400: loss = 3.96034836769104
step = 6396600: loss = 4.10599422454834
step = 6396800: loss = 2.1380767822265625
ste

step = 6430400: loss = 4.28941535949707
step = 6430600: loss = 3.0479037761688232
step = 6430800: loss = 3.3355937004089355
step = 6431000: loss = 3.2052743434906006
step = 6431200: loss = 3.7274656295776367
step = 6431400: loss = 4.508556842803955
step = 6431600: loss = 3.806908369064331
step = 6431800: loss = 4.118042469024658
step = 6432000: loss = 3.809757709503174
step = 6432200: loss = 3.931835889816284
step = 6432400: loss = 3.9952192306518555
step = 6432600: loss = 3.942119598388672
step = 6432800: loss = 3.3144617080688477
step = 6433000: loss = 3.897374391555786
step = 6433200: loss = 4.517910480499268
step = 6433400: loss = 4.381270885467529
step = 6433600: loss = 3.4535162448883057
step = 6433800: loss = 3.6380908489227295
step = 6434000: loss = 3.9658761024475098
step = 6434200: loss = 4.309756755828857
step = 6434400: loss = 3.978519916534424
step = 6434600: loss = 4.292654514312744
step = 6434800: loss = 4.024550914764404
step = 6435000: loss = 2.922452688217163
step = 6

In [ ]:
def play(policy):
    agent_policy = policy
    pygame.init()
    running = True
    env = TsuroEnv()
    state = env.reset()
    
    for i in range(env.num_players):
        temp = np.where(env.player_board == i+1)
        print("PLAYER " + str(i+1) + "'S STARTING POS: \nTile: " + str(temp[0]) + "    Node: " + str(temp[1]))
        
    while running:
        to_move = True
        env.render(mode = "human")
        print(env.player_tiles[0])
        print(env.player_tiles[1])
        while to_move:
            mouse = pygame.mouse.get_pos()
            if env.current_player == 1:
                if 75 + 100 > mouse[0] > 75 and 635 + 100 > mouse[1] > 635:
                    for event in pygame.event.get():
                        if event.type == pygame.KEYDOWN:
                            if event.key == pygame.K_r:
                                tile = env.tiles[env.player_tiles[0][0]]
                                tile.rotate_tile(1)
                                env.render(mode = "human")
                        if event.type == pygame.MOUSEBUTTONDOWN:
                            to_move = False
                            env.step(move = 0)
                            env.render(mode = "human")
                elif 275 + 100 > mouse[0] > 275 and 635 + 100 > mouse[1] > 635:
                    for event in pygame.event.get():
                        if event.type == pygame.KEYDOWN:
                            if event.key == pygame.K_r:
                                tile = env.tiles[env.player_tiles[0][1]]
                                tile.rotate_tile(1)
                                env.render(mode = "human")
                        if event.type == pygame.MOUSEBUTTONDOWN:
                            to_move = False
                            env.step(move = 1)
                            env.render(mode = "human")
                elif 475 + 100 > mouse[0] > 475 and 635 + 100 > mouse[1] > 635:
                    for event in pygame.event.get():
                        if event.type == pygame.KEYDOWN:
                            if event.key == pygame.K_r:
                                tile = env.tiles[env.player_tiles[0][2]]
                                tile.rotate_tile(1)
                                env.render(mode = "human")
                        if event.type == pygame.MOUSEBUTTONDOWN:
                            to_move = False
                            env.step(move = 2)
                            env.render(mode = "human")

            elif env.current_player == 2:
                time.sleep(2)
                observation = env.get_state
                action = agent_policy.action(observation)
                env.step(action)
                env.render(mode = "human")
                
            if env.current_player == -1 or env.game_is_over():
                env.render(mode = "human")
                print("Winner: Player " + str(env.current_player))
                running = False

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                    pygame.quit()
                 
    if not running:
        env.render(mode = "human")
        
    while not running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()


In [ ]:
iterations = range(0, train_step_counter + 1, eval_interval)
plt.plot(iterations, returns)
plt.ylabel('Average Return')
plt.xlabel('Iterations')
plt.ylim(top=10)